# Домашнее задание № 5
Дисциплина - Python для инженерии данных  
Тема 5. Классы и объекты (ООП)

In [39]:
import pandas as pd
from datetime import datetime
from re import *

Класс исключений

In [10]:
class ValidationError(Exception):
    def __init__(self, msg):
        self.message = msg

    def __str__(self):
        return self.message

### Задание 1  
Реализуйте базовый класс `Account`, который моделирует поведение банковского счёта. Этот класс должен не только выполнять базовые операции, но и вести детальный учёт всех действий, а также предоставлять аналитику по истории операций.

#### Этап 1. Реализация базового класса Account  
Класс должен быть инициализирован с параметрами:  
- account_holder (str) — имя владельца счёта;  
- balance (float, по умолчанию 0) — начальный баланс счёта, не может быть отрицательным.
  
Атрибуты:  
- _account_counter — приватный атрибут для хранения количества созданных счетов. Отсчет начинается с 1000;
- holder — хранит имя владельца;  
- account_number — хранит номер счёта;  
- _balance — приватный атрибут для хранения текущего баланса;  
- operations_history — список или другая структура для хранения истории операций.  
  
__Важно__:  
Каждая операция должна храниться не просто как число, а как структурированная информация, например, словарь, кортеж или класс. Минимальный набор данных для операции: тип операции ('deposit' или 'withdraw'), сумма, дата и время операции, текущий баланс после операции, статус ('success' или 'fail').

#### Этап 2. Реализация методов  
1. `__init__(self, account_holder, balance=0)` — конструктор. Обратите внимание, что в конструкторе должен автоматически формироваться номер счёта в формате ‘ACC-XXXX’, где XXXX — порядковый номер
счёта;

2. `deposit(self, amount)` — метод для пополнения счёта:
- принимает сумму (должна быть положительной), попытка положить отрицательную сумму, должна вызывать исключение;  
- в случае успеха обновляет баланс и добавляет запись в историю операций.
  
3. `withdraw(self, amount)` — метод для снятия средств:  
- принимает сумму (должна быть положительной);  
- проверяет, достаточно ли средств на счёте, если нет — операция не проходит, но ее попытка с статусом 'fail' все равно фиксируется в истории;  
- в случае успеха обновляет баланс и добавляет запись в историю.
  
4. `get_balance(self)` — метод, который возвращает текущий баланс.
   
5. `get_history(self)` — метод, который возвращает историю операций.Важно: продумайте, в каком формате его вернуть. Для работы с датой и временем используйте модуль `datetime`. Получить текущее время можно с помощью `datetime.now()`.

#### Этап 3. Визуализация истории операций.  
1. Создайте метод `plot_history(self)`, который использует библиотеку Pandas для создания датафрейма из истории операций.  
2. Продумайте, с помощью какой библиотеки можно отобразить изменение баланса с течением времени. Постройте простой линейный график, где по оси X будет время операции, а по оси Y — баланс после каждой операции. График должен иметь заголовок, подписи осей.

In [43]:
def process_date(date, time):
    if date is None:
        date = datetime.now().date()
        time = datetime.now().time()
        return datetime.now().date(), datetime.now().time()
    elif time is None:
        date = datetime.strptime(date, "%Y/%m/%d").date()
        time = datetime.strptime("12:00:00", "%H:%M:%S").time() 
    else:
        date = datetime.strptime(date, "%Y/%m/%d").date()
        time = datetime.strptime(time, "%H:%M:%S").time() 
    return date, time


class Account:
    def __init__(self, account_holder, balance=0):
        self._account_counter = 'ACC-1000'
        self.holder = account_holder
        self._balance = balance
        self.operations_history = []

    def deposit(self, amount, date=None, time=None):
        date, time = process_date(date, time)
        try:
            if amount < 0:
                raise ValidationError("Вносимая на счёт сумма не должна быть отрицательной!")
            self._balance += amount
            self.operations_history.append({"type": 'deposit', 
                                            'amount': amount, 
                                            "date": date,
                                            "time": time,
                                            "balance:": self._balance, 
                                            "status:": 'success'})
        except ValidationError as e:
            print(e)            

    def withdraw(self, amount, date=None, time=None):
        date, time = process_date(date, time)
        try:
            if amount < 0:
                raise ValidationError("Снимаемая со счёта сумма не должна быть отрицательной!")
            if amount > self._balance:
                self.operations_history.append({"type": 'withdraw', 
                                'amount': amount, 
                                "date": datetime.now().date(),
                                "time": datetime.now().time(),
                                "balance:": self._balance, 
                                "status:": 'fail'})
                raise ValidationError("На счету недостаточно средств для совершения этой операции!")
            self._balance -= amount
            self.operations_history.append({"type": 'withdraw', 
                                            'amount': amount, 
                                            "date": datetime.now().date(),
                                            "time": datetime.now().time(),
                                            "balance:": self._balance, 
                                            "status:": 'success'})
        except ValidationError as e:
            print(e)
            

    def get_balance(self):
        return self._balance

    def get_history(self):
        return self.operations_history
    

Тестирование класса

In [12]:
tester = Account("Tester")
tester.deposit(2000)
tester.withdraw(-20)
tester.withdraw(1000)
tester.withdraw(3000)
tester.deposit(200)
tester.withdraw(300)
print(tester.get_balance())
for entry in tester.get_history():
    print(entry)

Снимаемая со счёта сумма не должна быть отрицательной!
На счету недостаточно средств для совершения этой операции!
900
{'type': 'deposit', 'amount': 2000, 'date': datetime.date(2025, 11, 8), 'time': datetime.time(23, 10, 59, 95299), 'balance:': 2000, 'status:': 'success'}
{'type': 'withdraw', 'amount': 1000, 'date': datetime.date(2025, 11, 8), 'time': datetime.time(23, 10, 59, 95299), 'balance:': 1000, 'status:': 'success'}
{'type': 'withdraw', 'amount': 3000, 'date': datetime.date(2025, 11, 8), 'time': datetime.time(23, 10, 59, 96273), 'balance:': 1000, 'status:': 'fail'}
{'type': 'deposit', 'amount': 200, 'date': datetime.date(2025, 11, 8), 'time': datetime.time(23, 10, 59, 96273), 'balance:': 1200, 'status:': 'success'}
{'type': 'withdraw', 'amount': 300, 'date': datetime.date(2025, 11, 8), 'time': datetime.time(23, 10, 59, 96273), 'balance:': 900, 'status:': 'success'}


### Задание 2  

#### Этап 4. Реализация наследования
1. Реализуйте два класса `CheckingAccount` (расчётный счёт) и `SavingsAccount` (сберегательный счёт), которые отражают абстракцию базового поведения банковских аккаунтов:
- наследуются от базового класса `Account`;  
- хранят атрибут класса `account_type`.  

2. Класс `SavingsAccount` (сберегательный счёт) дополнительно должен реализовывать метод расчёта процентов на остаток `apply_interest(self, rate)` (например, 7% на остаток).  

3. Класс `SavingsAccount` (сберегательный счёт) позволяет снимать деньги только до определенного порога баланса: нельзя снять больше 50% от баланса. Переопределите метод снятия со счёта.

4. Реализуйте валидацию на отрицательные суммы и корректность имени владельца:  
- имя владельца счёта должно быть в формате «Имя Фамилия» с заглавных букв, кириллицей или латиницей, иначе — должно вызываться исключение;  
- попытка положить отрицательную сумму должна вызывать исключение.

5. Реализуйте метод для анализа истории транзакций по размеру и дате:
- метод должен выводить последние n крупных операций.

In [70]:
class CheckingAccount(Account):
    def __init__(self, account_holder, balance=0):
        try:
            super().__init__(account_holder, balance=balance)
            self.account_type = 'Checking'
            if not match(r'^[A-ZА-ЯЁ][a-zа-яё]* [A-ZА-ЯЁ][a-zа-яё]*$', self.holder):
                raise ValidationError("Неверный формат имени!")
            if self._balance < 0:
                raise ValidationError("Стартовый баланс не может быть отрицательным!")
        except ValidationError as e:
            print(e)

    def apply_interest(self, rate=7):
        self._balance = round(self._balance * (100 + rate) / 100, 2)

class SavingsAccount(Account):
    def __init__(self, account_holder, balance=0):
        try:
            super().__init__(account_holder, balance=balance)
            self.account_type = 'SavingsAccount'
            if not match(r'^[A-ZА-ЯЁ][a-zа-яё]* [A-ZА-ЯЁ][a-zа-яё]*$', self.holder):
                raise ValidationError("Неверный формат имени!")
            if self._balance < 0:
                raise ValidationError("Стартовый баланс не может быть отрицательным!")
        except ValidationError as e:
            print(e)

    def withdraw(self, amount, date=None, time=None):
        date, time = process_date(date, time)
        try:
            if amount < 0:
                raise ValidationError("Снимаемая со счёта сумма не должна быть отрицательной!")
            if amount > self._balance or 2 * amount > self._balance:
                self.operations_history.append({"type": 'withdraw', 
                                'amount': amount, 
                                "date": date,
                                "time": time,
                                "balance:": self._balance, 
                                "status:": 'fail'})
                error_type = ["На счету недостаточно средств для совершения этой операции!",
                              "Нельзя снять больше 50% от баланса!"]
                raise ValidationError(error_type[int(2 * amount > self._balance)])
            
            self._balance -= amount
            self.operations_history.append({"type": 'withdraw', 
                                            'amount': amount, 
                                            "date": date,
                                            "time": time,
                                            "balance:": self._balance, 
                                            "status:": 'success'})
        except ValidationError as e:
            print(e)

    def major_operations_history(self, type="by_amount", min_sum = 10e5, count=10):
        if type == "by_amount":
            return list(filter(lambda x: x["amount"] >= min_sum, self.operations_history))[-count:]
        if type == "last_month":
            return list(filter(lambda x: (datetime.now().date() - x["date"]).days >= 31, 
                               self.operations_history))[-count:]

Тестирование классов

In [73]:
tester = SavingsAccount("Dumbledore")
tester = SavingsAccount("Ron Weasley", -10)
tester = SavingsAccount("Harry Potter", 40_000)
tester.withdraw(30_000, "2020/01/01")
for i in range(1, 5):
    tester.withdraw(i * 1000, f"2025/11/0{i}")
# tester.get_history()
n = 3000
print(f"Операции, превышающие {n}:")
print(*tester.major_operations_history(min_sum=n), sep="\n")

tester = CheckingAccount("Hermione Granger", 1000)
tester.deposit(500)
for _ in range(10):
    tester.apply_interest()
print(tester.get_balance())

Неверный формат имени!
Стартовый баланс не может быть отрицательным!
Нельзя снять больше 50% от баланса!
Операции, превышающие 3000:
{'type': 'withdraw', 'amount': 30000, 'date': datetime.date(2020, 1, 1), 'time': datetime.time(12, 0), 'balance:': 40000, 'status:': 'fail'}
{'type': 'withdraw', 'amount': 3000, 'date': datetime.date(2025, 11, 3), 'time': datetime.time(12, 0), 'balance:': 34000, 'status:': 'success'}
{'type': 'withdraw', 'amount': 4000, 'date': datetime.date(2025, 11, 4), 'time': datetime.time(12, 0), 'balance:': 30000, 'status:': 'success'}
2950.73


In [7]:
data = pd.read_csv('transactions_dirty.csv')
data.head(40)

,account_number,account_type,date,operation,amount,balance_after,status
0,ACC-100001,checking,2025-09-27 22:17:26,deposit,921.00,2121.00,success
1,ACC-100001,checking,2025-09-27 22:17:26,deposit,607.00,2728.00,success
2,ACC-100001,checking,2025-09-28 22:17:26,deposit,488.00,3216.00,success
3,ACC-100001,checking,28/09/2025 22:17,deposit,129.00,3345.00,success
4,ACC-100001,checking,2025-09-29 22:17:26,deposit,880.00,4225.00,success
5,ACC-100001,checking,2025-09-29 22:17:26,withdraw,NaN,4039.00,success
6,ACC-100001,checking,2025-10-01 22:17:26,withdraw,352.00,3687.00,success
7,ACC-100001,checking,2025-10-01 22:17:26,withdraw,65.00,3622.00,success
8,ACC-100001,checking,2025-10-01 22:17:26,NaN,654.00,4276.00,success
9,ACC-100001,checking,2025-10-01 22:17:26,withdraw,245.00,4031.00,success
